In [2]:
import os
os.chdir('/fs01/home/sidarya/PDDM_MEGA/bayesian_dpddm')
import torch
import numpy as np

SEED = 9927
np.random.seed(SEED)
torch.random.manual_seed(SEED)

In [3]:
!which python

/h/sidarya/.conda/envs/bayesian_pddm/bin/python


In [4]:
from data.data_utils import UCIDataset
data_dict = torch.load('data/uci_data/uci_heart_torch.pt')
uci_dict = {}
for k, data in data_dict.items():
    data = list(zip(*data))
    X, y = torch.stack(data[0]), torch.tensor(data[1], dtype=torch.int)
    
    # normalize
    min_ = torch.min(X, dim=0).values
    max_ = torch.max(X, dim=0).values
    X = (X - min_) / (max_ - min_)
    uci_dict[k] = UCIDataset(X, y)

In [5]:
dataset_dict = {}
dataset_dict['train'] = uci_dict['train']
dataset_dict['valid'] = uci_dict['val']
dataset_dict['dpddm_train'] = uci_dict['val']
dataset_dict['dpddm_id'] = uci_dict['iid_test']
dataset_dict['dpddm_ood'] = uci_dict['ood_test']

In [6]:
%load_ext autoreload
%autoreload 2


In [7]:
import hydra
from omegaconf import DictConfig
from bayesian_dpddm import MLPModel, DPDDMBayesianMonitor, DPDDMFullInformationMoniter
from bayesian_dpddm.monitors import DPDDMFullInformationMoniter
from experiments.utils import get_configs

In [8]:
from hydra.core.global_hydra import GlobalHydra

if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()

In [9]:
hydra.initialize(config_path='./PDDM_MEGA/bayesian_dpddm/experiments/configs', version_base='1.2')
args = hydra.compose(config_name="uci")

In [10]:
model_config, train_config = get_configs(args)


In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = MLPModel(model_config, train_size=len(dataset_dict['train']))

monitor = DPDDMFullInformationMoniter(
    model=base_model,
    trainset=dataset_dict['train'],
    valset=dataset_dict['valid'],
    train_cfg=train_config,
    device=device,
)

In [45]:
output_metrics = monitor.train_model(tqdm_enabled=True)

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:00<00:43,  1.13it/s]

Epoch:  0, train loss: 0.6346
Epoch:  0, valid loss: 0.5643


 22%|██▏       | 11/50 [00:09<00:34,  1.13it/s]

Epoch: 10, train loss: 0.4854
Epoch: 10, valid loss: 0.4638


...

 62%|██████▏   | 31/50 [00:27<00:16,  1.13it/s]

Epoch: 30, train loss: 0.4414
Epoch: 30, valid loss: 0.4323


 82%|████████▏ | 41/50 [00:36<00:08,  1.11it/s]

Epoch: 40, train loss: 0.4801
Epoch: 40, valid loss: 0.4311


100%|██████████| 50/50 [00:44<00:00,  1.13it/s]


In [13]:
monitor.output_metrics["train_acc"]

[np.float64(0.561814695596695),
 np.float64(0.7446546057860056),
 np.float64(0.765213817358017),
 np.float64(0.773026317358017),
 np.float64(0.7754934231440226),
 np.float64(0.757538378238678),
 np.float64(0.7606907884279887),
 np.float64(0.7635690768559774),
 np.float64(0.7705592115720113),
 np.float64(0.804413378238678),
 np.float64(0.7876918911933899),
 np.float64(0.8079769710699717),
 np.float64(0.7668585578600565),
 np.float64(0.7841282884279887),
 np.float64(0.789610743522644),
 np.float64(0.7913925449053446),
 np.float64(0.788788378238678),
 np.float64(0.7982456187407175),
 np.float64(0.7720668911933899),
 np.float64(0.797423243522644),
 np.float64(0.7982456187407175),
 np.float64(0.8071546057860056),
 np.float64(0.8078399101893107),
 np.float64(0.8087993462880453),
 np.float64(0.7948190768559774),
 np.float64(0.8078399101893107),
 np.float64(0.781661182641983),
 np.float64(0.7824835578600565),
 np.float64(0.7868695159753164),
 np.float64(0.8060581187407175),
 np.float64(0.80167

In [47]:
monitor.pretrain_disagreement_distribution(dataset=dataset_dict['dpddm_train'],
                                           n_post_samples=args.dpddm.n_post_samples,
                                           data_sample_size=args.dpddm.data_sample_size,
                                           Phi_size=args.dpddm.Phi_size, 
                                           temperature=args.dpddm.temp,
                                           )

100%|██████████| 500/500 [05:51<00:00,  1.42it/s]


In [49]:
stats = {}
dis_rates = {}

for k,dataset in {
    'dpddm_train': dataset_dict['dpddm_train'],
    'dpddm_id': dataset_dict['dpddm_id'],
    'dpddm_ood': dataset_dict['dpddm_ood']
}.items():
    rate, max_dis_rates = monitor.repeat_tests(n_repeats=args.dpddm.n_repeats,
                                    dataset=dataset, 
                                    n_post_samples=args.dpddm.n_post_samples,
                                    data_sample_size=args.dpddm.data_sample_size,
                                    temperature=args.dpddm.temp
                                    )
    print(f"{k}: {rate}")
    stats[k] = rate
    dis_rates[k] = (np.mean(max_dis_rates), np.std(max_dis_rates))

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [01:09<00:00,  1.44it/s]


dpddm_train: 0.23


100%|██████████| 100/100 [01:09<00:00,  1.43it/s]


dpddm_id: 0.1


100%|██████████| 100/100 [01:10<00:00,  1.42it/s]

dpddm_ood: 0.9
